# TF-IDF
---

In [5]:
import csv
from konlpy.tag import Kkma
from math import log2, log10
import nltk
import pickle
import re
import sqlite3

In [6]:
section_list = ['society', 'politics', 'economic', 'culture', 'digital', 'global']
section_dict = {'society':'사회', 'politics':'정치', 'economic':'경제',
               'culture':'문화', 'digital':'IT', 'global':'세계'}

## Extract Nouns and Index Articles
---

* Section별로 명사 추출


* 기본 불용어 제거
   * 공통
      * 길이가 1인 단어
      * 숫자만으로 이루어진 단어
   * Section별
      * ?


* Inverted indexing


* Section별로 저장

In [7]:
def is_number(obj):
    try:
        float(obj)
        return True
    except ValueError:
        return False

In [17]:
def inverted_index(db_name, table_name, sections=['society', 'politics', 'economic', 'culture', 'digital', 'global'], write=False):
    k = Kkma()
    
    conn = sqlite3.connect('db/' + db_name + '.db')
    cur = conn.cursor()

    for section in sections:
        print(section_dict[section])
        print('--------------------------------------------------\n')

        cur.execute("SELECT a_ids, contents FROM {0} WHERE sections = '{1}'".format(table_name, section_dict[section]))

        contents = cur.fetchall()
        
        unique_nouns = []
        a_nouns = {}
        noun_max_cnt = {}
        inverted_idx = {}

        for content_idx, content in enumerate(contents):
            if (content_idx % 100) == 0:
                print('{0:6,} / {1:6,}'.format(content_idx, len(contents)))

            a_id = content[0]
            a_nouns[a_id] = {}
            nouns_cnt = {}
            noun_max_cnt[a_id] = 0

            # 연속된 공백 및 개행 제거
            content = re.sub(r'[\s]{2,}', ' ', content[1])
            content = re.sub(r'[\n]{2,}', '\n', content)

            # 문장 단위 토큰화
            for idx, sentence in enumerate(nltk.sent_tokenize(content)):
                nouns_temp = []

        #         print(idx)
        #         print('------------------------------')
        #         print(sentence.strip())

                # 단어 단위 토큰화
                for word in nltk.word_tokenize(sentence.strip()):
                    # 명사 추출
                    nouns = k.nouns(word)
                    
                    stop_words = []

                    for noun in nouns:
                        # 기본 불용어 선별 (공통)
                        # 길이가 1인 단어
                        if len(noun) == 1:
                            stop_words.append(noun)
                            
                            continue
                        # 숫자만으로 이루어진 단어
                        elif is_number(noun):
                            stop_words.append(noun)
                            
                            continue

                        # 기본 불용어 선별 (섹션별)
                        # TODO:
#                         if section == 'society':
#                             pass
#                         elif section == 'politics':
#                             pass
#                         elif section == 'economic':
#                             pass
#                         elif section == 'culture':
#                             pass
#                         elif section == 'digital':
#                             pass
#                         elif section == 'global':
#                             pass

                        if noun in nouns_cnt.keys():
                            nouns_cnt[noun] += 1
                        else:
                            nouns_cnt[noun] = 1

                        if noun_max_cnt[a_id] < nouns_cnt[noun]:
                            noun_max_cnt[a_id] = nouns_cnt[noun]
                    
                    # 기본 불용어 제거
                    for stop_word in stop_words:
                        nouns.remove(stop_word)

                    nouns_temp.extend(nouns)
                    nouns_temp = list(set(nouns_temp))

                unique_nouns.extend(nouns_temp)
                unique_nouns = list(set(unique_nouns))
                
#                 print('------------------------------\n\n')

            a_nouns[a_id] = nouns_cnt

        for noun in unique_nouns:
            inverted_idx[noun] = []

            for a_id, nouns in a_nouns.items():
                if noun in nouns:
                    inverted_idx[noun].append(a_id)
                    
        print('Start to write files.')
    
        if write:
            with open('index/' + section + '_unique_nouns.csv', 'w', newline='') as f:
                csv_writer = csv.writer(f)
                csv_writer.writerow(unique_nouns)

            with open('index/' + section + '_article_nouns.csv', 'w', newline='') as f:
                csv_writer = csv.writer(f)
                for a_id, nouns in a_nouns.items():
                    csv_writer.writerow([a_id, nouns])
#                     for noun in nouns:
#                         csv_writer.writerow([a_id, noun])

#             with open('index/' + section + '_noun_max_count.csv', 'w', newline='') as f:
#                 csv_writer = csv.writer(f)
#                 for a_id, max_cnt in noun_max_cnt.items():
#                     csv_writer.writerow([a_id, max_cnt])

            with open('index/' + section + '_inverted_index.csv', 'w', newline='') as f:
                csv_writer = csv.writer(f)
                for noun, a_ids in inverted_idx.items():
                    csv_writer.writerow([noun, a_ids])

            with open('index/' + section + '_unique_nouns.pkl', 'wb') as f:
                pickle.dump(unique_nouns, f)

            with open('index/' + section + '_article_nouns.pkl', 'wb') as f:
                pickle.dump(a_nouns, f)

            with open('index/' + section + '_noun_max_count.pkl', 'wb') as f:
                pickle.dump(noun_max_cnt, f)
                
            with open('index/' + section + '_inverted_index.pkl', 'wb') as f:
                pickle.dump(inverted_idx, f)

        print('\n--------------------------------------------------\n\n')

In [77]:
inverted_index('daum', 'daum', write=True)
# inverted_index('daum', 'daum', sections=['politics'], write=True)

사회
--------------------------------------------------

     0 /    227
   100 /    227
   200 /    227
Start to write files.

--------------------------------------------------


정치
--------------------------------------------------

     0 /    257
   100 /    257
   200 /    257
Start to write files.

--------------------------------------------------


경제
--------------------------------------------------

     0 /    229
   100 /    229
   200 /    229
Start to write files.

--------------------------------------------------


문화
--------------------------------------------------

     0 /    199
   100 /    199
Start to write files.

--------------------------------------------------


IT
--------------------------------------------------

     0 /    237
   100 /    237
   200 /    237
Start to write files.

--------------------------------------------------


세계
--------------------------------------------------

     0 /     65
Start to write files.

---------------------------

## [Debug] Extract Nouns and Index Articles
---

In [19]:
# 정치
for section in section_list[1:2]:
    with open('index/' + section + '_unique_nouns.pkl', 'rb') as f:
        unique_nouns = pickle.load(f)
    
    with open('index/' + section + '_article_nouns.pkl', 'rb') as f:
        a_nouns = pickle.load(f)
        
    with open('index/' + section + '_noun_max_count.pkl', 'rb') as f:
        noun_max_cnt = pickle.load(f)
        
    with open('index/' + section + '_inverted_index.pkl', 'rb') as f:
        inverted_idx = pickle.load(f)

In [20]:
conn = sqlite3.connect('db/daum.db')
cur = conn.cursor()

In [21]:
cur.execute("SELECT a_ids, contents FROM daum WHERE sections = '정치'")

a_id, content = cur.fetchone()

print(a_id, '\n')
print(content)

da_20180822211258392 

6·13 지방선거에 바른미래당 서울시장 후보로 나섰다가 참패한 뒤 독일 등 해외에 머물겠다고 했던 안철수 전 의원이 서울 마포구에서 포착됐다.
아주경제는 지난 21일 마포구의 한 사무실에서 기자와 마주치자 도망치는 안 전 의원의 모습(사진)을 22일 공개했다. 이 매체가 공개한 동영상에는 안 전 의원이 기자를 피해 황급히 계단을 내려가는 장면이 담겼다. 기자가 “죄를 지으신 게 아니지 않느냐”며 거듭 취재를 요청했지만 안 전 의원은 일절 답하지 않고 계단 아래쪽으로 뛰어 내려갔다.
지난달 12일 안 전 의원은 기자회견을 열고 “정치 일선에서 물러나 통찰과 채움의 시간을 갖고자 한다”며 “대한민국이 당면한 시대적 난제를 앞서 해결하고 있는 독일에서 해결의 실마리를 얻겠다”고 말했었다. 그런 그가 당대표 선거가 한창인 지금 서울에 머물고 있는 게 의아하다는 반응도 나온다.
안 전 의원이 서울에서 포착됐다는 보도에 대해 이준석 바른미래당 당대표 후보는 페이스북에 “이런 상황에서 음험한 계략을 꾸미는 분이 아니고 도망가실 분도 아니다. 그냥 바쁘셔서 그러셨을 거다”라고 썼다.


In [22]:
unique_nouns.sort()

len(unique_nouns)

8198

In [37]:
unique_nouns[0:10]

['0.2㎜',
 '0.4㎜',
 '0.6㎜',
 '0.86배',
 '0.95배',
 '08월',
 '1.43배',
 '1.4배',
 '1.5배',
 '10.5㎜']

In [24]:
len(a_nouns)

257

In [47]:
for a_id, nouns in a_nouns.items():
    print(a_id)
    print('Max count: ', noun_max_cnt[a_id])
    print('------------------------------\n')
    
    keys = list(nouns.keys())
    keys.sort()
    
    for k in keys:
        # 두 번 이상 나온 단어만 출력
        if nouns[k] >= 2:
            print('{0} : {1:,}'.format(k, nouns[k]))
        else:
#             print('{0} : {1:,}'.format(k, nouns[k]))
            pass
    
    print('\n------------------------------\n\n')
    
    break

da_20180822211258392
Max count:  6
------------------------------

계단 : 2
공개 : 2
기자 : 3
당대표 : 2
대표 : 2
독일 : 2
마포 : 2
마포구 : 2
미래 : 2
미래당 : 2
서울 : 4
의원 : 6
포착 : 2
해결 : 2
후보 : 2

------------------------------




In [48]:
cnt = 0

for noun, a_ids in inverted_idx.items():
    if cnt == 10:
        break
    
    # 전체 기사에서 10번 이상 나온 단어만 출력 (10개)
    if len(a_ids) >= 10:
        print('{0} : {1:,}'.format(noun, len(a_ids)))
        
        cnt += 1
    else:
#         print('{0} : {1:,}'.format(noun, len(a_ids)))
        pass

10시 : 11
제안 : 13
기관 : 35
조카 : 10
박자 : 10
압박 : 19
여동생 : 13
국방 : 26
방안 : 35
마련 : 28


## 불용어 제거
---

In [ ]:
def NewsStopWord(word):
    try:
        int(word) #숫자일경우
    except:        
        if type(word) is str and word.__contains__('일'):
            return True
        if type(word) is str and word.__contains__('회차'):
            return True
        if len(word) == 1:
            # 한글자 빠짐
            return True
        if re.search(r'\d{4}.\d{1,2}',word) != None:
            # 2018.12... 
            return True
        if re.match(r'\d{1,2}월',word) != None:
            return True
        if re.match(r'\d{1,2}년',word) != None:
            return True
        
        newsDic = {"기자":1,"배포":1,"금지":1,"뉴스":1,"저작권자":1,
                   "기사":1,"전재":1,"무단":1,"무단전재":1,"구독":1,"기사보기":1}
        
        pressDic = {'연합뉴스':1,'뉴시스':1,'뉴시스통신사':1,'통신사':1,
                    '이데일리':1,'네이버':1,'다음':1,'시스':1,'뉴스1':1,'뉴스1코리':1}
        
        nothingDic = {'사진':1,'페이스북':1,'관련':1,'웹툰보기':1,'가기':1,'만큼':1,
                     '최근':1,'재인':1,'올해':1,'시간':1,'판단':1,'추진':1,'우리':1,'반영':1,
                      '상황':1,'호텔':1,'운영':1,'주요':1,'적극':1,'대상':1,'때문':1,
                      '확인':1,'가능':1,'이야기':1,'규모':1,'개월':1,'종합':1,'위원회':1,
                      '가운데':1,'분석':1,'다양':1,'문제':1,'기간':1,'마련':1,'지난해':1,'신청':1,'한편':1,'기준':1,
                      '내용':1,'채널설정':1,'경우':1,'방안':1,'활용':1,'여러분':1,'기존':1,'최대':1,'스냅':1,'오전':1,'대비':1,
                      '위원','지난달':1,'이번달':1,'다음달':1,'위원장':1,'센터':1,'포함':1,'등에':1,'사진영상부':1,
                      '구성':1,'수준':1,'기대':1,'공동':1,'안내':1,'활동':1,'첫날':1,'추가':1,'분야':1,'관리':1,
                      '동안':1,'이용':1,'모습':1,'오늘':1,
                     }
        """  
        논의, 입장, 업계, 내년, 블록, 체인, 실시간, 고객, 채널, 보기, 오후, 이번, 이날, 진행, 제공, 예정, 연합, 대표, 제보, 이상, 지원, 행사, 관계자, 설정, 계획, 단체, 타임, 이후, 발표 
        """
        if word in newsDic.keys() or word in pressDic.keys() or word in nothingDic.keys():
            return True
        return False
    else:
        return True

## TF-IDF
---

* Parameters

In [49]:
k_ratio = 0.5

In [50]:
a_tf = {}

for a_id, nouns in a_nouns.items():
    tf_temp = {}
    
    for noun in nouns:
        # Double normalization K
        tf_temp[noun] = k_ratio + (1-  k_ratio) * (nouns[noun] / noun_max_cnt[a_id])
        
#         print("{0} | {1} + {2} * ({3} / {4}) = {5}".format(noun, k_ratio, (1 - k_ratio), nouns[noun], noun_max_cnt[a_id], tf_temp[noun]))
    
    a_tf[a_id] = tf_temp
    
#     break

len(a_tf)

257

In [59]:
# a_tf[tuple(a_tf.keys())[0]]

In [52]:
a_size = len(a_tf)
a_tfidf = {}

for noun, a_ids in inverted_idx.items():
    for a_id in a_ids:
        tfidf_temp = {}
        
        for noun in a_tf[a_id]:
            tfidf_temp[noun] = a_tf[a_id][noun] * log10(a_size / len(a_ids))
        
#             print("{0} | {1} * log({2} / {3}) = {4}".format(noun, a_tf[a_id][noun], a_size, len(a_ids), tfidf_temp[noun]))
        
        a_tfidf[a_id] = tfidf_temp
        
#         break
        
len(a_tfidf)

257

In [60]:
# a_tfidf[tuple(a_tfidf.keys())[0]]

## [Debug] TF-IDF
---

In [57]:
cur.execute("SELECT a_ids, contents FROM daum WHERE sections = '정치'")

a_id, content = cur.fetchone()

print(a_id, '\n')
print(content)

da_20180822211258392 

6·13 지방선거에 바른미래당 서울시장 후보로 나섰다가 참패한 뒤 독일 등 해외에 머물겠다고 했던 안철수 전 의원이 서울 마포구에서 포착됐다.
아주경제는 지난 21일 마포구의 한 사무실에서 기자와 마주치자 도망치는 안 전 의원의 모습(사진)을 22일 공개했다. 이 매체가 공개한 동영상에는 안 전 의원이 기자를 피해 황급히 계단을 내려가는 장면이 담겼다. 기자가 “죄를 지으신 게 아니지 않느냐”며 거듭 취재를 요청했지만 안 전 의원은 일절 답하지 않고 계단 아래쪽으로 뛰어 내려갔다.
지난달 12일 안 전 의원은 기자회견을 열고 “정치 일선에서 물러나 통찰과 채움의 시간을 갖고자 한다”며 “대한민국이 당면한 시대적 난제를 앞서 해결하고 있는 독일에서 해결의 실마리를 얻겠다”고 말했었다. 그런 그가 당대표 선거가 한창인 지금 서울에 머물고 있는 게 의아하다는 반응도 나온다.
안 전 의원이 서울에서 포착됐다는 보도에 대해 이준석 바른미래당 당대표 후보는 페이스북에 “이런 상황에서 음험한 계략을 꾸미는 분이 아니고 도망가실 분도 아니다. 그냥 바쁘셔서 그러셨을 거다”라고 썼다.


In [76]:
print('\tTF\tIDF')
for noun in a_tf[a_id].keys():
    print('{0}:\n\t{1:6.3} {2:6.4}'.format(noun, a_tf[a_id][noun], a_tfidf[a_id][noun]))

	TF	IDF
지방:
	 0.583  1.406
지방선거:
	 0.583  1.406
선거:
	 0.583  1.406
미래:
	 0.667  1.607
미래당:
	 0.667  1.607
서울:
	 0.833  2.008
서울시장:
	 0.583  1.406
시장:
	 0.583  1.406
후보:
	 0.667  1.607
참패:
	 0.583  1.406
독일:
	 0.667  1.607
해외:
	 0.583  1.406
안철수:
	 0.583  1.406
철수:
	 0.583  1.406
의원:
	   1.0   2.41
마포:
	 0.667  1.607
마포구:
	 0.667  1.607
포착:
	 0.667  1.607
경제:
	 0.583  1.406
21일:
	 0.583  1.406
사무실:
	 0.583  1.406
기자:
	  0.75  1.807
모습:
	 0.583  1.406
사진:
	 0.583  1.406
22일:
	 0.583  1.406
공개:
	 0.667  1.607
동영상:
	 0.583  1.406
피해:
	 0.583  1.406
계단:
	 0.667  1.607
장면:
	 0.583  1.406
취재:
	 0.583  1.406
요청:
	 0.583  1.406
아래쪽:
	 0.583  1.406
지난달:
	 0.583  1.406
12일:
	 0.583  1.406
기자회견:
	 0.583  1.406
회견:
	 0.583  1.406
정치:
	 0.583  1.406
일선:
	 0.583  1.406
통찰:
	 0.583  1.406
시간:
	 0.583  1.406
대한:
	 0.583  1.406
대한민국:
	 0.583  1.406
민국:
	 0.583  1.406
당면:
	 0.583  1.406
시대적:
	 0.583  1.406
난제:
	 0.583  1.406
해결:
	 0.667  1.607
실마리:
	 0.583  1.406
당대표:
	 0.667  1.607
대표:
	 0.667  1.607
한창

## ?
---

In [189]:
def merge(arrA, arrB, ary):
    iA = 0;
    iB = 0;
    idx = 0;

    while iA < len(arrA):
        if iB < len(arrB):
            if arrA[iA] < arrB[iB]:
                ary[idx] = arrA[iA]
                iA+=1
            else:
                ary[idx] = arrB[iB]
                iB+=1
            idx+=1

        else:
            while iA < len(arrA):
                ary[idx] = arrA[iA]
                iA+=1
                idx+=1

    while iB < len(arrB):
        ary[idx] = arrB[iB]
        iB+=1
        idx+=1

def mergeSort(ary):
    n = len(ary)
    if n==1: return;
    
    ary_temp1 = {}
    ary_temp2 = {}
    
    for i in range(int(n/2)):
        for k,v in ary.items():
            ary_temp1[k] = v

    for i in range(n-int(n/2)):
        for k,v in ary.items():
            ary_temp1[k] = v
    
    mergeSort(ary_temp1);
    mergeSort(ary_temp2);

    merge(ary_temp1, ary_temp2, ary)